In [3]:
import tensorflow as tf
from tensorflow import keras

In [4]:
# install keras tuner
!pip install -q -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 4.7 MB/s eta 0:00:00


In [5]:
# import keras tuner
import keras_tuner as kt

In [6]:
# load dataset
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [7]:
label_train

array([9, 0, 0, ..., 3, 0, 5], dtype=uint8)

In [8]:
label_test

array([9, 2, 1, ..., 8, 1, 5], dtype=uint8)

In [9]:
img_train[0]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1,
          0,   0,  13,  73,   0,   0,   1,   4,   0,   0,   0,   0,   1,
          1,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,
          0,  36, 136, 127,  62,  54,   0,   0,   0,   1,   3,   4,   0,
          0,   3],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   6,
          0, 102, 204, 176, 134, 144, 123,  23,   0,   0,   0,   0,  12,
         10,   0],
       [  

In [10]:
#black and white pixels ranges from 0 to 255

In [11]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [12]:
img_train[0][20]

array([0.29411766, 0.8       , 0.83137256, 0.8       , 0.75686276,
       0.8039216 , 0.827451  , 0.88235295, 0.84705883, 0.7254902 ,
       0.77254903, 0.80784315, 0.7764706 , 0.8352941 , 0.9411765 ,
       0.7647059 , 0.8901961 , 0.9607843 , 0.9372549 , 0.8745098 ,
       0.85490197, 0.83137256, 0.81960785, 0.87058824, 0.8627451 ,
       0.8666667 , 0.9019608 , 0.2627451 ], dtype=float32)

In [13]:
def model_builder(hp):
  model = keras.Sequential()
  #input layer
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32) #int for range
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]) #choice for option

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [14]:
#RandomSearch, Hyperband, BayesianOptimization, Sklearn - 4types of tuners
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [15]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [16]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 00m 51s]
val_accuracy: 0.8835833072662354

Best val_accuracy So Far: 0.8934166431427002
Total elapsed time: 00h 13m 26s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 416 and the optimal learning rate for the optimizer
is 0.001.



In [17]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 7s 4ms/step - loss: 0.4965 - accuracy: 0.8231 - val_loss: 0.4140 - val_accuracy: 0.8487
Epoch 2/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3707 - accuracy: 0.8656 - val_loss: 0.3734 - val_accuracy: 0.8651
Epoch 3/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3334 - accuracy: 0.8770 - val_loss: 0.3622 - val_accuracy: 0.8710
Epoch 4/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3090 - accuracy: 0.8859 - val_loss: 0.3378 - val_accuracy: 0.8780
Epoch 5/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2895 - accuracy: 0.8934 - val_loss: 0.3185 - val_accuracy: 0.8862
Epoch 6/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2704 - accuracy: 0.8988 - val_loss: 0.3417 - val_accuracy: 0.8762
Epoch 7/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.2597 - accuracy: 0.9037 - val_loss: 0.3334 - val_accuracy:

In [19]:
hypermodel = tuner.hypermodel.build(best_hps)

eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 2ms/step - loss: 2.5159 - accuracy: 0.0532
[test loss, test accuracy]: [2.515916585922241, 0.053199999034404755]
